In [1]:
!pip install -q transformers datasets peft bitsandbytes accelerate wandb trl scipy

# Verify GPU
!nvidia-smi

# Fix seeds for reproducibility
import torch, random, numpy as np
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

# Log versions
import transformers, datasets, peft
print(f"transformers: {transformers.__version__}")
print(f"datasets: {datasets.__version__}")
print(f"peft: {peft.__version__}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 30.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 85.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 35.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 53.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/

2025-11-04 03:19:33.240577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762226373.408248      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762226373.454920      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


transformers: 4.57.1
datasets: 4.1.1
peft: 0.16.0


In [2]:
  from datasets import load_dataset
  import random
  import numpy as np

  # Set seed for reproducibility
  random.seed(42)
  np.random.seed(42)

  # Load GSM8K
  dataset = load_dataset("gsm8k", "main")

  # Keep test set UNTOUCHED for final evaluation
  test_set = dataset["test"]  # 1,319 examples - DO NOT USE YET

  # Split train (7,473) into train/val (90/10)
  train_val = dataset["train"].train_test_split(test_size=0.1, seed=42)
  train_set = train_val["train"]  # ~6,726 examples
  val_set = train_val["test"]     # ~747 examples

  print(f"Train: {len(train_set)}, Val: {len(val_set)}, Test: {len(test_set)}")
  print("\nSample from training set:")
  print(train_set[0])

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Train: 6725, Val: 748, Test: 1319

Sample from training set:
{'question': 'A kilogram of pork costs $6 while a kilogram of chicken costs $2 less. How much will a 3-kilogram of chicken and a kilogram of pork cost?', 'answer': 'A kilogram of chicken costs $6 - $2 = $<<6-2=4>>4.\nThree kilograms of chicken cost $4 x 3 = $<<4*3=12>>12.\nSo, a 3-kilogram of chicken and a kilogram of pork cost $12 + $6 = $18.\n#### 18'}


In [3]:

def format_gsm8k_example(example):
  """
  Format: 
  - System prompt enforcing ≤12 steps
  - Numbered atomic steps
  - FINAL_ANSWER: <number>
  """
  question = example["question"]
  solution = example["answer"]  # Original solution

  # Extract ground truth answer (after ####)
  ground_truth = solution.split("####")[-1].strip()

  # Parse solution into atomic steps (simplified - you'll refine this)
  steps = solution.split("\n")
  steps = [s.strip() for s in steps if s.strip() and "####" not in s]

  # Format as numbered steps (limit to 12)
  formatted_steps = "\n".join([f"{i+1}) {step}" for i, step in enumerate(steps[:12])])

  # Create training example
  prompt = f"""Solve the following math problem step-by-step. Use at most 12 numbered steps. End with FINAL_ANSWER: <number>.

Problem:
{question}

Solution:"""

  response = f"""{formatted_steps}
FINAL_ANSWER: {ground_truth}"""

  return {
      "prompt": prompt,
      "response": response,
      "question": question,
      "answer": ground_truth
  }

# Apply formatting
train_formatted = train_set.map(format_gsm8k_example)
val_formatted = val_set.map(format_gsm8k_example)



def validate_example(example):
  """Ensure quality: ≤12 steps, has FINAL_ANSWER, numeric answer"""
  response = example["response"]

  # Check for FINAL_ANSWER
  if "FINAL_ANSWER:" not in response:
      return False

  # Count steps
  step_count = len([line for line in response.split("\n") if line.strip() and line[0].isdigit()])
  if step_count > 12:
      return False

  # Check numeric answer exists
  answer = response.split("FINAL_ANSWER:")[-1].strip()
  if not any(char.isdigit() for char in answer):
      return False

  return True

# Filter invalid examples
train_formatted = train_formatted.filter(validate_example)
val_formatted = val_formatted.filter(validate_example)

Map:   0%|          | 0/6725 [00:00<?, ? examples/s]

Map:   0%|          | 0/748 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6725 [00:00<?, ? examples/s]

Filter:   0%|          | 0/748 [00:00<?, ? examples/s]

In [4]:

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

# Optimized QLoRA for T4 (fp16)
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.float16,  # fp16 for T4
)

# Load Phi-2
model_name = "microsoft/phi-2"
model = AutoModelForCausalLM.from_pretrained(
  model_name,
  quantization_config=bnb_config,
  device_map="auto",
  trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Prepare for training
model = prepare_model_for_kbit_training(model)

# Optimized LoRA: r=16, alpha=16
lora_config = LoraConfig(
  r=16,
  lora_alpha=16,
  lora_dropout=0.1,
  bias="none",
  target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
  task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

print("\n✅ Model loaded with optimized LoRA (r=16, alpha=16)")

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

trainable params: 7,864,320 || all params: 2,787,548,160 || trainable%: 0.2821

✅ Model loaded with optimized LoRA (r=16, alpha=16)


In [5]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
import torch

# Make sure the tokenizer has a pad token
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

MAX_LEN = 1024

def format_and_tokenize(batch):
  """
  Tokenize and mask prompt tokens so loss is only computed on the response.
  This is the standard approach for instruction fine-tuning.
  """
  inputs = []
  attn = []
  labels = []

  for p, r in zip(batch["prompt"], batch["response"]):
      # Tokenize full text (prompt + response)
      full = p + " " + r
      tok_full = tokenizer(full, truncation=True, max_length=MAX_LEN, padding=False)

      # Tokenize just the prompt to find where it ends
      tok_prompt = tokenizer(p, truncation=True, max_length=MAX_LEN, padding=False)

      # Create labels: mask prompt tokens with -100 (ignored in loss)
      lab = tok_full["input_ids"][:]
      prompt_len = len(tok_prompt["input_ids"])
      lab[:prompt_len] = [-100] * prompt_len

      inputs.append(tok_full["input_ids"])
      attn.append(tok_full["attention_mask"])
      labels.append(lab)

  return {"input_ids": inputs, "attention_mask": attn, "labels": labels}

print("Tokenizing datasets...")
tokenized_train = train_formatted.map(
  format_and_tokenize,
  batched=True,
  remove_columns=train_formatted.column_names,
  desc="Tokenizing training set"
)
tokenized_val = val_formatted.map(
  format_and_tokenize,
  batched=True,
  remove_columns=val_formatted.column_names,
  desc="Tokenizing validation set"
)

print(f"✅ Tokenized {len(tokenized_train)} training examples")
print(f"✅ Tokenized {len(tokenized_val)} validation examples")

# ✅ FIXED: Use DataCollatorForSeq2Seq instead of default_data_collator
data_collator = DataCollatorForSeq2Seq(
  tokenizer=tokenizer,
  model=model,
  padding=True,
  pad_to_multiple_of=8,
)

training_args = TrainingArguments(
  output_dir="./generator_checkpoints",
  num_train_epochs=2,
  per_device_train_batch_size=1,
  gradient_accumulation_steps=32,
  learning_rate=1e-4,
  lr_scheduler_type="cosine",
  warmup_ratio=0.03,
  fp16=True,
  gradient_checkpointing=True,
  max_grad_norm=1.0,

  eval_strategy="steps",
  eval_steps=200,
  logging_steps=50,
  save_strategy="steps",
  save_steps=200,
  save_total_limit=2,

  report_to="none",
  load_best_model_at_end=True,
  metric_for_best_model="loss",
)

trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_train,
  eval_dataset=tokenized_val,
  data_collator=data_collator,  # ← Using the fixed collator
)

print("\n" + "="*60)
print("TRAINING CONFIGURATION")
print("="*60)
print(f"Training examples: {len(tokenized_train)}")
print(f"Validation examples: {len(tokenized_val)}")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"Total epochs: {training_args.num_train_epochs}")
print(f"Estimated steps per epoch: ~{len(tokenized_train) // 32}")
print(f"Total training steps: ~{(len(tokenized_train) // 32) * 2}")
print("="*60)

# ✅ Resume from checkpoint to save time!



trainer.train()

print("\n✅ Training complete!")

# Save final model
trainer.save_model("./generator_final")
tokenizer.save_pretrained("./generator_final")
print("✅ Model saved to ./generator_final")

Tokenizing datasets...


Tokenizing training set:   0%|          | 0/6725 [00:00<?, ? examples/s]

Tokenizing validation set:   0%|          | 0/748 [00:00<?, ? examples/s]

✅ Tokenized 6725 training examples
✅ Tokenized 748 validation examples

TRAINING CONFIGURATION
Training examples: 6725
Validation examples: 748
Effective batch size: 32
Total epochs: 2
Estimated steps per epoch: ~210
Total training steps: ~420


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
200,0.540900,0.493366
400,0.525200,0.486181



✅ Training complete!
✅ Model saved to ./generator_final


In [12]:
  print("Train columns:", train_formatted.column_names)
  print("\nFirst example keys:", train_formatted[0].keys())
  print("\nSample data:")
  print(train_formatted[0])

Train columns: ['question', 'answer', 'prompt', 'response']

First example keys: dict_keys(['question', 'answer', 'prompt', 'response'])

Sample data:
{'question': 'A kilogram of pork costs $6 while a kilogram of chicken costs $2 less. How much will a 3-kilogram of chicken and a kilogram of pork cost?', 'answer': '18', 'prompt': 'Solve the following math problem step-by-step. Use at most 12 numbered steps. End with FINAL_ANSWER: <number>.\n\nProblem:\nA kilogram of pork costs $6 while a kilogram of chicken costs $2 less. How much will a 3-kilogram of chicken and a kilogram of pork cost?\n\nSolution:', 'response': '1) A kilogram of chicken costs $6 - $2 = $<<6-2=4>>4.\n2) Three kilograms of chicken cost $4 x 3 = $<<4*3=12>>12.\n3) So, a 3-kilogram of chicken and a kilogram of pork cost $12 + $6 = $18.\nFINAL_ANSWER: 18'}


In [11]:

  from peft import PeftModel
  from transformers import AutoModelForCausalLM, AutoTokenizer
  import torch
  import re
  from tqdm import tqdm

  # Load trained model
  print("Loading trained model...")
  base_model_eval = AutoModelForCausalLM.from_pretrained(
      "microsoft/phi-2",
      device_map="auto",
      trust_remote_code=True,
      dtype=torch.float16,
  )
  model_eval = PeftModel.from_pretrained(base_model_eval, "./generator_final")
  model_eval.eval()
  tokenizer_eval = AutoTokenizer.from_pretrained("./generator_final", trust_remote_code=True)
  print("✅ Model loaded\n")

  def generate_solution(model, tokenizer, question, max_new_tokens=512):
      """Generate with deterministic decoding"""
      prompt = f"""Solve the following math problem step-by-step. Use at most 12 numbered steps. End with FINAL_ANSWER: <number>.

  Problem:
  {question}

  Solution:"""

      inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

      with torch.no_grad():
          outputs = model.generate(
              **inputs,
              max_new_tokens=max_new_tokens,
              temperature=0.0,
              do_sample=False,
              pad_token_id=tokenizer.eos_token_id,
          )

      generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
      solution = generated.split("Solution:")[-1].strip()

      return solution

  def extract_answer(solution_text):
      """Extract numeric answer after FINAL_ANSWER:"""
      match = re.search(r'FINAL_ANSWER:\s*([^\n]+)', solution_text)
      if not match:
          return None

      answer_str = match.group(1).strip()
      answer_str = re.sub(r'[$,]', '', answer_str)

      num_match = re.search(r'-?\d+\.?\d*', answer_str)
      if num_match:
          return float(num_match.group())

      return None

  def evaluate_pass_at_1(model, tokenizer, dataset, show_examples=5):
      """Evaluate Pass@1 accuracy"""
      correct = 0
      total = 0
      invalid_format = 0

      for i, example in enumerate(tqdm(dataset, desc="Evaluating")):
          question = example["question"]

          ground_truth_str = str(example["answer"])
          ground_truth_str = re.sub(r'[$,]', '', ground_truth_str)

          try:
              ground_truth = float(ground_truth_str)
          except:
              continue

          solution = generate_solution(model, tokenizer, question)
          predicted = extract_answer(solution)

          if predicted is None:
              invalid_format += 1
              continue

          is_correct = abs(predicted - ground_truth) < 1e-4
          if is_correct:
              correct += 1

          total += 1

          if i < show_examples:
              print(f"\n{'='*60}")
              print(f"Example {i+1}:")
              print(f"Q: {question[:100]}...")
              print(f"Ground Truth: {ground_truth}")
              print(f"Predicted: {predicted}")
              print(f"Correct: {'✅' if is_correct else '❌'}")
              print(f"Solution:\n{solution[:400]}...")

      accuracy = correct / total if total > 0 else 0
      format_rate = (total / len(dataset)) * 100

      return {
          "accuracy": accuracy,
          "correct": correct,
          "total": total,
          "format_rate": format_rate,
          "invalid_format": invalid_format
      }

  print("="*60)
  print("EVALUATION: Pass@1 on Validation Set (748 examples)")
  print("="*60)

  results = evaluate_pass_at_1(model_eval, tokenizer_eval, val_formatted)

  print(f"\n{'='*60}")
  print("FINAL RESULTS - MILESTONE 1")
  print("="*60)
  print(f"Pass@1 Accuracy: {results['accuracy']*100:.2f}%")
  print(f"Correct: {results['correct']}/{results['total']}")
  print(f"Valid Format Rate: {results['format_rate']:.2f}%")
  print(f"Invalid Format: {results['invalid_format']}")
  print("="*60)

  import json
  with open("eval_results.json", "w") as f:
      json.dump(results, f, indent=2)
  print("\n✅ Results saved to eval_results.json")
  print("✅ MILESTONE 1 COMPLETE!")

Loading trained model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model loaded

EVALUATION: Pass@1 on Validation Set (748 examples)


Evaluating:   0%|          | 1/748 [00:05<1:10:12,  5.64s/it]


Example 1:
Q: Mimi picked up 2 dozen seashells on the beach.  Kyle found twice as many shells as Mimi and put them...
Ground Truth: 16.0
Predicted: 16.0
Correct: ✅
Solution:
1) Mimi picked up 2 x 12 = <<2*12=24>>24 seashells.
2) Kyle found twice as many shells as Mimi, so he found 2 x 24 = <<2*24=48>>48 shells.
3) Leigh grabbed one-third of the shells that Kyle found, so she had 48 / 3 = <<48/3=16>>16 shells.
FINAL_ANSWER: 16...


...


Example 4:
Q: Emma's bank account has $100 in it. Each day of the week, she spends $8. At the end of the week, she...
Ground Truth: 4.0
Predicted: 24.0
Correct: ❌
Solution:
1) Emma spends $8 x 7 = $<<8*7=56>>56 in a week.
2) She has $100 - $56 = $<<100-56=44>>44 left in the account.
3) She can get $44 / $5 = <<44/5=8.8>>8.8 $5 bills.
4) She can get 8 $5 bills and $44 - 8*$5 = $<<44-8*5=24>>24 remain in the account.
FINAL_ANSWER: 24...


Evaluating:   1%|          | 5/748 [00:42<1:37:54,  7.91s/it]


Example 5:
Q: Ezekiel hikes as a hobby. This past summer, he did a challenging three-day hike across 50 kilometers...
Ground Truth: 15.0
Predicted: 15.0
Correct: ✅
Solution:
1) The second day was half the full hike distance, so he hiked 50/2 = <<50/2=25>>25 kilometers.
2) He hiked 10 + 25 = <<10+25=35>>35 kilometers on the first two days.
3) He had to hike 50 - 35 = <<50-35=15>>15 kilometers on the third day to finish the hike.
FINAL_ANSWER: 15...


Evaluating: 100%|██████████| 748/748 [1:25:00<00:00,  6.82s/it]


FINAL RESULTS - MILESTONE 1
Pass@1 Accuracy: 64.06%
Correct: 476/743
Valid Format Rate: 99.33%
Invalid Format: 5

✅ Results saved to eval_results.json
✅ MILESTONE 1 COMPLETE!


In [7]:
  print("Columns in val_formatted:", val_formatted.column_names)
  print("\nFirst example:")
  print(val_formatted[0])

Columns in val_formatted: ['question', 'answer', 'prompt', 'response']

First example:
{'question': 'Mimi picked up 2 dozen seashells on the beach.  Kyle found twice as many shells as Mimi and put them in his pocket. Leigh grabbed one-third of the shells that Kyle found.  How many seashells did Leigh have?', 'answer': '16', 'prompt': 'Solve the following math problem step-by-step. Use at most 12 numbered steps. End with FINAL_ANSWER: <number>.\n\nProblem:\nMimi picked up 2 dozen seashells on the beach.  Kyle found twice as many shells as Mimi and put them in his pocket. Leigh grabbed one-third of the shells that Kyle found.  How many seashells did Leigh have?\n\nSolution:', 'response': '1) Mimi has 2 x 12 = <<2*12=24>>24 sea shells.\n2) Kyle has 24 x 2 = <<24*2=48>>48 sea shells.\n3) Leigh has 48 / 3 = <<48/3=16>>16 sea shells.\nFINAL_ANSWER: 16'}


In [13]:



import shutil
import os

# Create a directory with essential files
os.makedirs("milestone1_deliverables", exist_ok=True)

# Copy generator_final
shutil.copytree("./generator_final", "./milestone1_deliverables/generator_final")

# Copy eval results
shutil.copy("eval_results.json", "./milestone1_deliverables/")

# Create a summary file
summary = f"""
MILESTONE 1 - Generator SLM Baseline
=====================================

Model: Phi-2 (2.7B) with QLoRA
Training: 2 epochs, ~2 hours
Pass@1 Accuracy: 64.06%
Correct: 476/743
Format Rate: 99.33%

Configuration:
- LoRA r=16, alpha=16
- Learning rate: 1e-4
- Batch size: 32 (effective)
- Max length: 1024
- Precision: FP16

Date: Nov 4, 2025
Team: SmolSolver
"""

with open("./milestone1_deliverables/SUMMARY.txt", "w") as f:
  f.write(summary)

# Create zip
shutil.make_archive("milestone1_deliverables", "zip", "./milestone1_deliverables")

print("✅ Created milestone1_deliverables.zip")
print("📦 Download this file (right-click → Download)")
print(f"📊 Size: {os.path.getsize('milestone1_deliverables.zip') / 1024 / 1024:.1f} MB")

✅ Created milestone1_deliverables.zip
📦 Download this file (right-click → Download)
📊 Size: 28.9 MB
